# 📓 01 Adquisición y Carga de Datos

Este notebook documenta el proceso de obtención de datos desde fuentes externas (OSM, IDE Chile) y su carga inicial en la base de datos PostGIS.

## 🛠️ Configuración

In [5]:
import os
import sys
from pathlib import Path
import geopandas as gpd
import osmnx as ox
from sqlalchemy import create_engine
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE RUTAS ---
# Añadimos el directorio raíz al PATH para permitir la importación de scripts locales si es necesario
project_root = Path("..")
sys.path.append(str(project_root))

# Cargamos las variables de entorno desde el archivo .env (credenciales de DB, API keys)
load_dotenv(project_root / ".env")

# --- CONEXIÓN A POSTGIS ---
# Construimos la URL de conexión usando las variables de entorno para mayor seguridad
db_url = (
    f"postgresql://{os.getenv('POSTGRES_USER')}:"
    f"{os.getenv('POSTGRES_PASSWORD')}@"
    f"{os.getenv('POSTGRES_HOST', 'localhost')}:5432/"
    f"{os.getenv('POSTGRES_DB')}"
)
engine = create_engine(db_url)
print("Conexión preparada.")

Conexión preparada.


## 1. Descarga de Datos

Utilizamos **OSMnx** para obtener los límites administrativos del área de estudio.

In [6]:
comuna = "San Bernardo, Chile"

# --- GEOCRIFICACIÓN Y DESCARGA ---
# ox.geocode_to_gdf permite obtener el polígono oficial de la comuna desde OpenStreetMap
print(f"Descargando límites para: {comuna}")
boundary = ox.geocode_to_gdf(comuna)

# Visualizamos el resultado inicial (GeoDataFrame)
boundary

## 2. Adquisición de Edificios y Amenidades

Descargamos las capas de infraestructura y equipamiento.

In [ ]:
# --- DESCARGA DE EQUIPAMIENTO (AMENITIES) ---
# Definimos las etiquetas de OSM que nos interesan para el análisis territorial
tags = {'amenity': True, 'landuse': ['commercial', 'industrial', 'retail']}
amenities = ox.features_from_place(comuna, tags=tags)

# --- DESCARGA DE HUELLA DE EDIFICIOS (BUILDINGS) ---
buildings = ox.features_from_place(comuna, tags={'building': True})

print(f"Amenidades encontradas: {len(amenities)}")
print(f"Edificios encontrados: {len(buildings)}")

## 3. Carga a PostGIS

Cargamos los datos procesados al esquema `raw_data` para centralizar la información.

In [7]:
# --- CARGA SISTEMÁTICA A LA DB ---
# Usamos to_postgis para inyectar los GeoDataFrames directamente en tablas espaciales
# schema='raw_data' organiza nuestros datos originales sin procesar

boundary.to_postgis("comuna_boundaries", engine, schema="raw_data", if_exists="replace")
print("Límites cargados exitosamente.")

# Nota: Para amenities y buildings, es recomendable limpiar columnas de listas antes de cargar
amenities = amenities.apply(lambda b: b.astype(str) if isinstance(b.dtype, object) else b)
amenities.to_postgis("osm_amenities", engine, schema="raw_data", if_exists="replace")

buildings = buildings.apply(lambda b: b.astype(str) if isinstance(b.dtype, object) else b)
buildings.to_postgis("osm_buildings", engine, schema="raw_data", if_exists="replace")

print("Carga completa terminada.")

Límites cargados exitosamente.
